In [1]:
!pip install --quiet llama-index-core llama-index-utils-workflow llama-index-llms-openai llama-index-graph-stores-neo4j 

In [1]:
import os
from llama_index.llms.openai import OpenAI
from neo4j.exceptions import CypherSyntaxError

from llama_index.core import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Literal, Union, Optional

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username="recommendations",
    password="recommendations",
    database="recommendations",
    url="neo4j+s://demo.neo4jlabs.com:7687",
    enhanced_schema=True,
    create_indexes=False
)

In [3]:
os.environ["OPENAI_API_KEY"] = "sk-"
llm = OpenAI(model="gpt-4o", temperature=0)

In [4]:
class Guardrail(BaseModel):
    """A restaurant with name, city, and cuisine."""

    decision: Literal["movie", "end"] = Field(
        description="Decision on whether the question is related to movies"
        
    )
guardrails_system_prompt = """As an intelligent assistant, your primary objective is to decide whether a given question is related to movies or not.
If the question is related to movies, output "movie". Otherwise, output "end".
To make this decision, assess the content of the question and determine if it refers to any movie, actor, director, film industry,
or related topics. Provide only the specified output: "movie" or "end"."""
# Refine Prompt
chat_refine_msgs = [
    (
        "system",
        guardrails_system_prompt,
    ),
    ("user", "The question is: {question}"),
]
guardrails_template = ChatPromptTemplate.from_messages(chat_refine_msgs)

In [5]:
class SubqueriesOutput(BaseModel):
    """Defines the output format for transforming a question. 
    The transformation ensures actionable results by either:
    1) Returning the original question if it is already specific and answerable, or
    2) Breaking it into a list of subquestions that comprehensively address the query's intent.
    Subquestions are designed to be distinct and directly usable for information retrieval."""
    
    subqueries: List[str] = Field(description=("""The processed output of the transformation. This field contains:
        1) The original question, unmodified, if it is specific and directly answerable.
        2) A list of clear, actionable subquestions if the original question is complex, broad, or multi-step.
        Subquestions must be relevant, non-redundant, and enable independent retrieval of specific information.
        Avoid including the original question within the list of subquestions."""
        ))


subqueries_system = """You are a question transformer. Your task is to analyze a given question and determine if it can be answered directly or if it should be broken down into relevant subquestions. The goal is to ensure actionable, efficient, and non-redundant information retrieval.

- **Specific, Answerable Questions**: If the question is specific and can be answered as is, return the original question without modification.  
- **Broad or Complex Questions**: If the question is complex or requires multiple steps to answer, decompose it into a list of distinct, actionable subquestions.

**Important**:
- Exclude subquestions that cannot currently be operationalized (e.g., questions lacking sufficient context or information to be actionable).  
- Subquestions should focus only on aspects of the original query that can directly facilitate progress.  
- Ensure that subquestions are distinct, non-redundant, and relevant to the final answer.  

**Output Requirements**:
- Return the **original question** if no subquestions are necessary or feasible.  
- If subquestions are required, return only a list of **actionable, specific subquestions**, ensuring no subquestion relies on unresolved dependencies.
"""
query_decompose_msgs = [
    (
        "system",
        subqueries_system,
    ),
    ("user", "{question}"),
]

subquery_template = ChatPromptTemplate.from_messages(query_decompose_msgs)

In [6]:
def guardrails_step(question):
    guardrails_output = (
        llm.as_structured_llm(Guardrail)
        .complete(guardrails_template.format(question=question))
        .raw
    ).decision
    if guardrails_output == 'end':
        context = "The question is not about movies or their case, so I cannot answer this question"
        return {"next_event": "generate_final_answer", "arguments": {"context": context, "question": question}}
    queries_output = (
        llm.as_structured_llm(SubqueriesOutput)
        .complete(subquery_template.format(question=question))
        .raw
    ).subqueries
    return {"next_event": "generate_cypher", "arguments": {"subqueries": queries_output, "question": question}}


In [7]:
guardrails_step("Who has appeared in more movies: Leonardo DiCaprio or the actor who has co-starred most frequently with the director of Tom Hanks' most critically acclaimed movie??")

{'next_event': 'generate_cypher',
 'arguments': {'subqueries': ['How many movies has Leonardo DiCaprio appeared in?',
   "Who is the director of Tom Hanks' most critically acclaimed movie?",
   'Who has co-starred most frequently with this director?',
   'How many movies has this actor appeared in?'],
  'question': "Who has appeared in more movies: Leonardo DiCaprio or the actor who has co-starred most frequently with the director of Tom Hanks' most critically acclaimed movie??"}}

In [8]:
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-small")


examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {title: 'Schindler's List'})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

few_shot_nodes = []
for line in examples:
    few_shot_nodes.append(TextNode(text=f"{{'query':{line['query']}, 'question': {line['question']}))"))

few_shot_index = VectorStoreIndex(few_shot_nodes, embed_model=embed_model)
few_shot_retriever = few_shot_index.as_retriever(similarity_top_k=5)


def get_fewshots(question):
    return [el.text for el in few_shot_retriever.retrieve(question)]

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
generate_system = """Given an input question, convert it to a Cypher query. No pre-amble.
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"""

generate_user = """You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!
Here is the schema information
{schema}

Below are a number of examples of questions and their corresponding Cypher queries.

{fewshot_examples}

User input: {question}
Cypher query:"""

generate_cypher_msgs = [
    (
        "system",
        generate_system,
    ),
    ("user", generate_user),
]

text2cypher_prompt = ChatPromptTemplate.from_messages(generate_cypher_msgs)

In [10]:
schema = graph_store.get_schema_str(exclude_types=["Actor", "Director"])

async def generate_cypher(subquery):
    fewshot_examples = get_fewshots(subquery)
    resp = await llm.achat(text2cypher_prompt.format_messages(question=subquery, schema=schema, fewshot_examples=fewshot_examples))
    return resp.message.content

In [11]:
validate_cypher_system = """
You are a Cypher expert reviewing a statement written by a junior developer.
"""

validate_cypher_user = """You must check the following:
* Are there any syntax errors in the Cypher statement?
* Are there any missing or undefined variables in the Cypher statement?
* Are any node labels missing from the schema?
* Are any relationship types missing from the schema?
* Are any of the properties not included in the schema?
* Does the Cypher statement include enough information to answer the question?

Examples of good errors:
* Label (:Foo) does not exist, did you mean (:Bar)?
* Property bar does not exist for label Foo, did you mean baz?
* Relationship FOO does not exist, did you mean FOO_BAR?

Schema:
{schema}

The question is:
{question}

The Cypher statement is:
{cypher}

Make sure you don't make any mistakes!"""

validate_cypher_msgs = [
    (
        "system",
        validate_cypher_system,
    ),
    ("user", validate_cypher_user),
]

validate_cypher_prompt = ChatPromptTemplate.from_messages(validate_cypher_msgs)

class Property(BaseModel):
    """
    Represents a filter condition based on a specific node property in a graph in a Cypher statement.
    """

    node_label: str = Field(
        description="The label of the node to which this property belongs."
    )
    property_key: str = Field(description="The key of the property being filtered.")
    property_value: str = Field(
        description="The value that the property is being matched against."
    )


class ValidateCypherOutput(BaseModel):
    """
    Represents the validation result of a Cypher query's output,
    including any errors and applied filters.
    """

    errors: Optional[List[str]] = Field(
        description="A list of syntax or semantical errors in the Cypher statement. Always explain the discrepancy between schema and Cypher statement"
    )
    filters: Optional[List[Property]] = Field(
        description="A list of property-based filters applied in the Cypher statement."
    )

In [12]:
from llama_index.graph_stores.neo4j import CypherQueryCorrector, Schema

# Cypher query corrector is experimental
corrector_schema = [
    Schema(el["start"], el["type"], el["end"])
    for el in graph_store.get_schema().get("relationships")
]
cypher_query_corrector = CypherQueryCorrector(corrector_schema)

In [13]:
def validate_cypher(question, cypher):
    """
    Validates the Cypher statements and maps any property values to the database.
    """
    errors = []
    mapping_errors = []
    # Check for syntax errors
    try:
        graph_store.structured_query(f"EXPLAIN {cypher}")
    except CypherSyntaxError as e:
        errors.append(e.message)
    # Experimental feature for correcting relationship directions
    corrected_cypher = cypher_query_corrector(cypher)
    if not corrected_cypher:
        errors.append("The generated Cypher statement doesn't fit the graph schema")
    # Use LLM to find additional potential errors and get the mapping for values
    llm_output =   (
        llm.as_structured_llm(ValidateCypherOutput)
        .complete(validate_cypher_prompt.format(question=question, cypher=cypher, schema=schema))
        .raw
    )
    print(f"LLM:{llm_output}")
    if llm_output.errors:
        errors.extend(llm_output.errors)
    if llm_output.filters:
        for filter in llm_output.filters:
            # Do mapping only for string values
            if (
                not [
                    prop
                    for prop in graph_store.get_schema()["node_props"][
                        filter.node_label
                    ]
                    if prop["property"] == filter.property_key
                ][0]["type"]
                == "STRING"
            ):
                continue
            print(f"Mapping: {filter}")
            mapping = graph_store.structured_query(
                f"MATCH (n:{filter.node_label}) WHERE toLower(n.`{filter.property_key}`) = toLower($value) RETURN 'yes' LIMIT 1",
                {"value": filter.property_value},
            )
            if not mapping:
                print(
                    f"Missing value mapping for {filter.node_label} on property {filter.property_key} with value {filter.property_value}"
                )
                mapping_errors.append(
                    f"Missing value mapping for {filter.node_label} on property {filter.property_key} with value {filter.property_value}"
                )
    if mapping_errors:
        next_action = "end"
    elif errors:
        next_action = "correct_cypher"
    else:
        next_action = "execute_cypher"

    return {
        "next_action": next_action,
        "cypher_statement": corrected_cypher,
        "cypher_errors": errors,
        "mapping_errors": mapping_errors,
        "steps": ["validate_cypher"],
    }

In [14]:
correct_cypher_system = """You are a Cypher expert reviewing a statement written by a junior developer. 
You need to correct the Cypher statement based on the provided errors. No pre-amble."
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"""

correct_cypher_user = """Check for invalid syntax or semantics and return a corrected Cypher statement.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not wrap the response in any backticks or anything else.
Respond with a Cypher statement only!

Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.

The question is:
{question}

The Cypher statement is:
{cypher}

The errors are:
{errors}

Corrected Cypher statement: """

# Correct cypher
correct_cypher_msgs = [
    (
        "system",
        correct_cypher_system,
    ),
    ("user", correct_cypher_user),
]

correct_cypher_prompt = ChatPromptTemplate.from_messages(generate_cypher_msgs)

In [15]:
async def correct_cypher(subquery, cypher, errors):
    resp = await llm.achat(correct_cypher_prompt.format_messages(question=subquery, schema=schema, errors=errors))
    return resp.message.content

In [16]:
information_check_system = """You are an expert assistant that evaluates whether a set of subqueries, their results, and any existing condensed information provide enough details to answer a given question. Your task is to determine if the available information is sufficient. If it is, summarize the information to directly answer the original question. If not, identify the missing pieces of information and suggest additional subqueries to obtain the needed data.

Follow this process:

1. Analyze the original question to understand the required information.
2. Review the subqueries, their results, and any provided condensed information to determine what has been addressed.
3. Check for gaps between the original question and the available information.
4. If sufficient information exists, generate a concise and accurate answer to the original question.
5. If insufficient, suggest additional subqueries to fill the gaps.

Ensure that all outputs are clear, precise, and actionable."""

information_check_user = """
Original question: {question}  
Subqueries and their results:  
{subqueries}  
Existing condensed information:  
{condensed_info}  
"""

information_check_msgs = [
    (
        "system",
        information_check_system,
    ),
    ("user", information_check_user),
]

information_check_prompt = ChatPromptTemplate.from_messages(information_check_msgs)

class IFOutput(BaseModel):
    """
    Represents the output of an information sufficiency evaluation process. 
    Contains either a condensed summary of the available information or additional subqueries needed to answer the original question.
    """

    condensed_information: str = Field(
        description="A summarized answer or information derived from the subquery results that directly addresses the original question."
    )
    additional_subqueries: Optional[List[str]] = Field(
        description="A list of additional subqueries suggested to gather missing information required to answer the original question."
    )

In [17]:
def format_subqueries_for_prompt(information_checks: list) -> str:
    """
    Converts a list of InformationCheck objects into a string that can be added to a prompt.
    
    Args:
        information_checks (List[InformationCheck]): List of information checks to process.
    
    Returns:
        str: A formatted string representing subqueries and their results.
    """
    subqueries_and_results = []
    
    for check in information_checks:
        # Extract the first result if available, otherwise use "No result available."
        result = (
            check.database_output[0] if check.database_output else "No result available."
        )
        subqueries_and_results.append(
            f"- Subquery: {check.subquery}\n  Result: {result}"
        )
    
    return "\n".join(subqueries_and_results)

def information_check(subquery_events, original_question, condensed_information):
    subqueries = format_subqueries_for_prompt(subquery_events)
    llm_output =   (
        llm.as_structured_llm(IFOutput)
        .complete(information_check_prompt.format(subqueries=subqueries, original_question=original_question, condensed_information=condensed_information))
        .raw
    )
    return {'condensed_information': llm_output.condensed_information, 'additional_subqueries': llm_output.additional_subqueries, 'ccondensed_information': condensed_information}
        

In [18]:
final_answer_system = """You are a highly intelligent assistant trained to provide concise and accurate answers. You will be given a context and a user question. Your task is to analyze the context and answer the user question based on the information provided in the context. If the context lacks sufficient information to answer the question, inform the user and suggest what additional details are needed.

Focus solely on the context to form your response. Avoid making assumptions or using external knowledge unless explicitly stated in the context.
Ensure the final answer is clear, relevant, and directly addresses the user’s question.
If the question is ambiguous, ask clarifying questions to ensure accuracy before proceeding."""

final_answer_user = """
Based on this context:
{context}

Answer the following question:
<question>
{question}
</question>

Provide your answer based on the context above explain your reasoning.
If clarification or additional information is needed, explain why and specify what is required.
"""

final_answer_msgs = [
    (
        "system",
        final_answer_system,
    ),
    ("user", final_answer_user),
]

final_answer_prompt = ChatPromptTemplate.from_messages(final_answer_msgs)

async def generate_final_answer(question, context):
    resp = await llm.achat(final_answer_prompt.format_messages(question=question, context=context))
    return resp.message.content                 
    

In [24]:
class GenerateCypher(Event):
    subquery: str
    
class ValidateCypher(Event):
    subquery: str
    generated_cypher: str

class CorrectCypher(Event):
    cypher: str
    subquery: str
    errors: List[str]

class ExecuteCypher(Event):
    validated_cypher: str
    subquery: str

class InformationCheck(Event):
    cypher: str
    subquery: str
    database_output: list
    
class GenerateFinalAnswer(Event):
    context: str

class ConcurrentFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> GenerateCypher | GenerateFinalAnswer:
        original_question = ev.input
        await ctx.set("original_question", original_question)
        await ctx.set("condensed_information", "")
        await ctx.set("subqueries_cypher", {})
        guardrails_output = guardrails_step(original_question)
        x = len(guardrails_output)
        print(guardrails_output)
        if guardrails_output.get("next_event") == "generate_final_answer":
            context = "The question is not about movies or cast, so I cannot answer the question"
            return GenerateFinalAnswer(context=context)

        # store in global context
        subqueries = guardrails_output["arguments"].get("subqueries")
        await ctx.set("count_of_subqueries", len(subqueries))
        # Send events
        for subquery in subqueries:
            ctx.send_event(GenerateCypher(subquery=subquery))

    @step(num_workers=4)
    async def generate_cypher_step(self, ctx: Context, ev: GenerateCypher) -> ValidateCypher:
        print("Running generate_cypher ", ev.subquery)
        generated_cypher = await generate_cypher(ev.subquery)
        return ValidateCypher(subquery=ev.subquery, generated_cypher=generated_cypher)

    @step(num_workers=4)
    async def validate_cypher_step(self, ctx: Context, ev: ValidateCypher) -> GenerateFinalAnswer | ExecuteCypher | CorrectCypher:
        print("Running validate_cypher ", ev)
        results = validate_cypher(ev.subquery, ev.generated_cypher)
        print(results)
        if results['next_action'] == "end": # DB value mapping
            return GenerateFinalAnswer(context=str(results["mapping_errors"]))
        if results['next_action'] == "execute_cypher":
            return ExecuteCypher(subquery=ev.subquery, validated_cypher=ev.generated_cypher)
        if results['next_action'] == "correct_cypher":
            return CorrectCypher(subquery=ev.subquery, cypher=ev.generated_cypher, errors=results['cypher_errors'])

    @step(num_workers=4)
    async def correct_cypher_step(self, ctx: Context, ev: CorrectCypher) -> ValidateCypher:
        print("Running validate_cypher ", ev)
        results = await correct_cypher(ev.subquery, ev.cypher, ev.errors)
        return ValidateCypher(subquery=ev.subquery, generated_cypher=results)
    
    @step
    async def execute_cypher_step(self, ctx: Context, ev: ExecuteCypher) -> InformationCheck:
        # wait until we receive all events
        print("Running execute_cypher_step ", ev)
        database_output = graph_store.structured_query(ev.validated_cypher)
        return InformationCheck(subquery=ev.subquery, cypher=ev.validated_cypher, database_output=database_output)

    @step
    async def information_check_step(self, ctx: Context, ev: InformationCheck) -> GenerateCypher | GenerateFinalAnswer:
        # wait until we receive all events
        print("Running information_check_step", ev)
        # retrieve from context
        number_of_subqueries = await ctx.get("count_of_subqueries")
        result = ctx.collect_events(ev, [InformationCheck] * number_of_subqueries)
        if result is None:
            return None
        # Add executed cypher statements to global state
        existing_subqueries_cypher = await ctx.get("subqueries_cypher")
        new_subqueries_cypher = {
                item.subquery: {
                    "cypher": item.cypher,
                    "database_output": item.database_output
                } for item in result
            }
        await ctx.set("subqueries_cypher", {**existing_subqueries_cypher, **new_subqueries_cypher})

        original_question = await ctx.get("original_question")
        condensed_information = await ctx.get("condensed_information")

        # Do the information check
        data = information_check(result, original_question, condensed_information)
        # Go fetch additional information if needed
        if data.get("additional_subqueries"):
            await ctx.set("count_of_subqueries", len(data['additional_subqueries']))
            condensed_information = await ctx.set("condensed_information", len(data["condensed_information"]))
            for subquery in data["additional_subqueries"]:
                ctx.send_event(GenerateCypher(subquery=subquery))
        else:
            return GenerateFinalAnswer(context=data['condensed_information'])

    @step
    async def final_answer(self, ctx: Context, ev: GenerateFinalAnswer) -> StopEvent:
        original_question = await ctx.get("original_question")
        subqueries_cypher = await ctx.get("subqueries_cypher")
        # wait until we receive all events
        print("Running final_answer ", ev)
        resp = await generate_final_answer(original_question, ev.context)
        return StopEvent(result={"text":resp, "subqueries_cypher": subqueries_cypher})

In [25]:
w = ConcurrentFlow(timeout=25, verbose=False)
result = await w.run(input="Who made more movies, Leonardo di Caprio or Tom Hanks?")
print(result)

{'next_event': 'generate_cypher', 'arguments': {'subqueries': ['How many movies has Leonardo DiCaprio acted in?', 'How many movies has Tom Hanks acted in?', 'Compare the number of movies Leonardo DiCaprio and Tom Hanks have acted in to determine who has made more.'], 'question': 'Who made more movies, Leonardo di Caprio or Tom Hanks?'}}
Running generate_cypher  How many movies has Leonardo DiCaprio acted in?
Running generate_cypher  How many movies has Tom Hanks acted in?
Running generate_cypher  Compare the number of movies Leonardo DiCaprio and Tom Hanks have acted in to determine who has made more.
Running validate_cypher  subquery='How many movies has Tom Hanks acted in?' generated_cypher="MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"
LLM:errors=None filters=[Property(node_label='Person', property_key='name', property_value='Tom Hanks')]
Mapping: node_label='Person' property_key='name' property_value='Tom Hanks'
{'next_action': 'execute_cypher', 'cyph

In [26]:
w = ConcurrentFlow(timeout=30, verbose=False)
result = await w.run(input="Who made more movies, Leonardo di Caprio or Tom Hanks most frequent coactor?")
print(result)

{'next_event': 'generate_cypher', 'arguments': {'subqueries': ['How many movies has Leonardo DiCaprio appeared in?', "Who is Tom Hanks' most frequent co-actor?", "How many movies has Tom Hanks' most frequent co-actor appeared in?"], 'question': 'Who made more movies, Leonardo di Caprio or Tom Hanks most frequent coactor?'}}
Running generate_cypher  How many movies has Leonardo DiCaprio appeared in?
Running generate_cypher  Who is Tom Hanks' most frequent co-actor?
Running generate_cypher  How many movies has Tom Hanks' most frequent co-actor appeared in?
Running validate_cypher  subquery='How many movies has Leonardo DiCaprio appeared in?' generated_cypher="MATCH (a:Person {name: 'Leonardo DiCaprio'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"
LLM:errors=None filters=[Property(node_label='Person', property_key='name', property_value='Leonardo DiCaprio')]
Mapping: node_label='Person' property_key='name' property_value='Leonardo DiCaprio'
{'next_action': 'execute_cypher', 'cypher_statement'

In [27]:
w = ConcurrentFlow(timeout=60, verbose=True)
result = await w.run(input="Who has appeared in more movies: Leonardo DiCaprio or the actor who has co-starred most frequently with the director of Tom Hanks' most critically acclaimed movie??")
print(result)

Running step start
{'next_event': 'generate_cypher', 'arguments': {'subqueries': ['How many movies has Leonardo DiCaprio appeared in?', "Who is the director of Tom Hanks' most critically acclaimed movie?", 'Who has co-starred most frequently with the director identified in the previous question?', 'How many movies has the actor identified in the previous question appeared in?'], 'question': "Who has appeared in more movies: Leonardo DiCaprio or the actor who has co-starred most frequently with the director of Tom Hanks' most critically acclaimed movie??"}}
Step start produced no event
Running step generate_cypher_step
Running generate_cypher  How many movies has Leonardo DiCaprio appeared in?
Running step generate_cypher_step
Running generate_cypher  Who is the director of Tom Hanks' most critically acclaimed movie?
Running step generate_cypher_step
Running generate_cypher  Who has co-starred most frequently with the director identified in the previous question?
Running step generate_c

In [28]:
w = ConcurrentFlow(timeout=30, verbose=False)
result = await w.run(input="What")
print(result)

{'next_event': 'generate_final_answer', 'arguments': {'context': 'The question is not about movies or their case, so I cannot answer this question', 'question': 'What'}}
Running final_answer  context='The question is not about movies or cast, so I cannot answer the question'
{'text': 'The context provided states that the question is not about movies or cast, but it does not give any specific information about what the question is actually about. The question "<question>What</question>" is incomplete and lacks context or detail. \n\nTo provide an accurate answer, I need more information about what the question is specifically asking. Please provide additional details or clarify the question so I can assist you better.', 'subqueries_cypher': {}}


In [ ]:
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)

draw_most_recent_execution(w, filename="joke_flow_recent.html")
draw_all_possible_flows(w, filename="joke_flow_recenst.html")

Task exception was never retrieved
future: <Task finished name='Task-46' coro=<Workflow.run.<locals>._run_workflow() done, defined at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/workflow/workflow.py:397> exception=InvalidStateError('invalid state')>
Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/workflow/workflow.py", line 445, in _run_workflow
    raise WorkflowTimeoutError(msg)
llama_index.core.workflow.errors.WorkflowTimeoutError: Operation timed out after 25 seconds

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/workflow/workflow.py", line 449, in _run_workflow
    result.set_exception(e)
asyncio.exceptions.InvalidStateError: invalid state
